<a href="https://colab.research.google.com/github/AchyuthNamburi/webDev/blob/main/Unet%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

kvasir_zip = '/content/drive/MyDrive/datasets/archive.zip'   # Kvasir
polydb_zip = '/content/drive/MyDrive/datasets/PolypDB.zip'   # PolyDB

!unzip -q "$kvasir_zip" -d ./kvasir/
!unzip -q "$polydb_zip" -d ./polydb/

print("Datasets extracted.")


Mounted at /content/drive
Datasets extracted.


In [2]:
!pip install opencv-python tqdm --quiet


In [3]:
import cv2, os, numpy as np
from tqdm import tqdm
from skimage import color

def clahe_lab(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l2 = clahe.apply(l)
    lab = cv2.merge((l2,a,b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

def preprocess_dataset(img_dir, mask_dir, save_img_dir, save_mask_dir, size=(256,256)):
    os.makedirs(save_img_dir, exist_ok=True)
    os.makedirs(save_mask_dir, exist_ok=True)

    images = [f for f in os.listdir(img_dir) if f.endswith(('.jpg','.png'))]

    for fname in tqdm(images, desc=f"Processing {img_dir}"):
        img_path  = os.path.join(img_dir, fname)
        mask_path = os.path.join(mask_dir, fname)

        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, 0)

        if img is None or mask is None:
            continue

        # ---- Image preprocessing
        img = cv2.resize(img, size)
        img = clahe_lab(img)
        img = cv2.GaussianBlur(img, (3,3), 0)

        # ---- Mask preprocessing
        mask = cv2.resize(mask, size)
        _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        cv2.imwrite(os.path.join(save_img_dir, fname), img)
        cv2.imwrite(os.path.join(save_mask_dir, fname), mask)


In [4]:
import os
import shutil
from glob import glob

# paths


kvasir_img = "/content/kvasir/Kvasir-SEG/Kvasir-SEG/images/"
kvasir_mask = "/content/kvasir/Kvasir-SEG/Kvasir-SEG/masks/"

polydb_root = "/content/polydb/PolypDB/PolypDB_modality_wise"

combined_img = "/content/drive/MyDrive/datasets/combined_dataset/images"
combined_mask = "/content/drive/MyDrive/datasets/combined_dataset/masks"

os.makedirs(combined_img, exist_ok=True)
os.makedirs(combined_mask, exist_ok=True)

# --- Copy Kvasir ---
for img_path in glob(os.path.join(kvasir_img, "*.jpg")):
    fname = "kvasir_" + os.path.basename(img_path)
    shutil.copy(img_path, os.path.join(combined_img, fname))

for mask_path in glob(os.path.join(kvasir_mask, "*.jpg")):
    fname = "kvasir_" + os.path.basename(mask_path)
    shutil.copy(mask_path, os.path.join(combined_mask, fname))

# --- Copy PolypDB (all modalities) ---
modalities = ["BLI", "FICE", "LCI", "NBI", "WLI"]
for mod in modalities:
    img_dir = os.path.join(polydb_root, mod, "images")
    mask_dir = os.path.join(polydb_root, mod, "masks")

    for img_path in glob(os.path.join(img_dir, "*.jpg")):
        fname = f"polydb_{mod}_" + os.path.basename(img_path)
        shutil.copy(img_path, os.path.join(combined_img, fname))

    for mask_path in glob(os.path.join(mask_dir, "*.jpg")):
        fname = f"polydb_{mod}_" + os.path.basename(mask_path)
        shutil.copy(mask_path, os.path.join(combined_mask, fname))

print("✅ Combined dataset created successfully!")


✅ Combined dataset created successfully!


In [8]:
# ✅ Kvasir dataset
kvasir_img_dir = "/content/kvasir/Kvasir-SEG/Kvasir-SEG/images/"
kvasir_mask_dir = "/content/kvasir/Kvasir-SEG/Kvasir-SEG/masks/"


# ✅ PolypDB dataset (modality-wise)
polydb_modality_dir = "/content/polydb/PolypDB/PolypDB_modality_wise/"

# ✅ Output combined dataset (you can change the name if you want)
merged_img_dir = "/content/combined_split/images/"
merged_mask_dir = "/content/combined_split/masks/"



In [9]:
import os
import shutil

# Ensure these variables are set before running
# kvasir_img_dir = your actual path
# kvasir_mask_dir = your actual path
# polydb_modality_dir = your actual path
# merged_img_dir = your actual path
# merged_mask_dir = your actual path

os.makedirs(merged_img_dir, exist_ok=True)
os.makedirs(merged_mask_dir, exist_ok=True)

counter = 0

# Merge Kvasir
for fname in sorted(os.listdir(kvasir_img_dir)):
    img_path = os.path.join(kvasir_img_dir, fname)
    mask_path = os.path.join(kvasir_mask_dir, fname)
    if os.path.exists(mask_path):
        new_name = f'kvasir_{counter:05d}.png'
        shutil.copy(img_path, os.path.join(merged_img_dir, new_name))
        shutil.copy(mask_path, os.path.join(merged_mask_dir, new_name))
        counter += 1

# Merge PolyDB modality-wise folders
for modality in os.listdir(polydb_modality_dir):
    modality_img_dir = os.path.join(polydb_modality_dir, modality, "images")
    modality_mask_dir = os.path.join(polydb_modality_dir, modality, "masks")
    if not os.path.exists(modality_img_dir) or not os.path.exists(modality_mask_dir):
        continue
    for fname in sorted(os.listdir(modality_img_dir)):
        img_path = os.path.join(modality_img_dir, fname)
        mask_path = os.path.join(modality_mask_dir, fname)
        if os.path.exists(mask_path):
            new_name = f'polydb_{modality}_{counter:05d}.png'
            shutil.copy(img_path, os.path.join(merged_img_dir, new_name))
            shutil.copy(mask_path, os.path.join(merged_mask_dir, new_name))
            counter += 1

print("Merge complete!")
print(f"Total images merged: {len(os.listdir(merged_img_dir))}")
print(f"Total masks merged: {len(os.listdir(merged_mask_dir))}")


Merge complete!
Total images merged: 1088
Total masks merged: 1088


In [10]:
# Kvasir
print("Kvasir images:", len(os.listdir(kvasir_img_dir)))
print("Kvasir masks:", len(os.listdir(kvasir_mask_dir)))

# PolyDB, count recursively per modality
import glob
for modality in os.listdir(polydb_modality_dir):
    img_path = os.path.join(polydb_modality_dir, modality, "images")
    mask_path = os.path.join(polydb_modality_dir, modality, "masks")
    n_imgs = len(glob.glob(os.path.join(img_path, '*')))
    n_masks = len(glob.glob(os.path.join(mask_path, '*')))
    print(f"{modality}: images={n_imgs}, masks={n_masks}")


Kvasir images: 1000
Kvasir masks: 1000
FICE: images=70, masks=70
LCI: images=60, masks=60
NBI: images=146, masks=146
BLI: images=70, masks=70
WLI: images=3588, masks=3588


In [13]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os, random, shutil
from sklearn.model_selection import train_test_split

merged_img_dir = "/content/drive/MyDrive/datasets/combined_dataset/images/"
merged_mask_dir = "/content/drive/MyDrive/datasets/combined_dataset/masks/"

split_base = "/content/combined_split2/"
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(split_base, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(split_base, split, "masks"), exist_ok=True)

# 1. Gather and shuffle all files
all_files = sorted(os.listdir(merged_img_dir))
random.shuffle(all_files)

# 2. Train/Val/Test Split (adjust ratios as needed)
trainval, test = train_test_split(all_files, test_size=0.10, random_state=42)  # 10% for test
train, val = train_test_split(trainval, test_size=0.1667, random_state=42)  # 16.67% of trainval ≈ 15%

# 3. Helper to copy files
def copy_split(files, split_name):
    for fname in files:
        shutil.copy(os.path.join(merged_img_dir, fname), os.path.join(split_base, split_name, "images", fname))
        shutil.copy(os.path.join(merged_mask_dir, fname), os.path.join(split_base, split_name, "masks", fname))

# 4. Move
copy_split(train, "train")
copy_split(val, "val")
copy_split(test, "test")

print(f"Data split complete. Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/datasets/combined_dataset/masks/polydb_NBI_d74c8c13-d28b-4e46-964f-54a8b78135c5.jpg'

In [16]:
import os, random, shutil
from sklearn.model_selection import train_test_split

merged_img_dir = "/content/drive/MyDrive/datasets/combined_dataset/images/"
merged_mask_dir = "/content/drive/MyDrive/datasets/combined_dataset/masks/"

split_base = "/content/combined_split2/"
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(split_base, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(split_base, split, "masks"), exist_ok=True)

# Ensure matching pairs only (check extensions)
all_imgs = sorted(os.listdir(merged_img_dir))
all_masks = sorted(os.listdir(merged_mask_dir))
pairs = []
for img in all_imgs:
    mask = img  # since you copied with same name for both
    if mask in all_masks:
        pairs.append(img)

print(f"Total pairs for splitting: {len(pairs)}")

# Shuffle and split
random.shuffle(pairs)
trainval, test = train_test_split(pairs, test_size=0.10, random_state=42)
train, val = train_test_split(trainval, test_size=0.1667, random_state=42)

def copy_split(file_list, split_name):
    for fname in file_list:
        shutil.copy(os.path.join(merged_img_dir, fname), os.path.join(split_base, split_name, "images", fname))
        shutil.copy(os.path.join(merged_mask_dir, fname), os.path.join(split_base, split_name, "masks", fname))

copy_split(train, "train")
copy_split(val, "val")
copy_split(test, "test")

print(f"Data split done. Train: {len(train)}, Val: {len(val)}, Test: {len(test)}")


Total pairs for splitting: 1086
Data split done. Train: 814, Val: 163, Test: 109


In [17]:
import os, random, shutil
from sklearn.model_selection import train_test_split

merged_img_dir = "/content/drive/MyDrive/datasets/combined_dataset/images/"
merged_mask_dir = "/content/drive/MyDrive/datasets/combined_dataset/masks/"

split_base = "/content/combined_split2/"

# Ensure splits exist
for split in ["train","val","test"]:
    os.makedirs(os.path.join(split_base, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(split_base, split, "masks"), exist_ok=True)

# Build pairs by matching filenames
image_files = sorted([f for f in os.listdir(merged_img_dir) if f.lower().endswith((".png",".jpg",".jpeg"))])
mask_files  = sorted([f for f in os.listdir(merged_mask_dir) if f.lower().endswith((".png",".jpg",".jpeg"))])

pairs = [f for f in image_files if f in mask_files]
print("Total valid pairs available:", len(pairs))

# Shuffle and split (adjust ratio if needed)
random.seed(42)
random.shuffle(pairs)

trainval, test = train_test_split(pairs, test_size=0.10, random_state=42)
train, val = train_test_split(trainval, test_size=0.1667, random_state=42)  # ~15% val

def copy_pair_list(file_list, src_img_dir, src_mask_dir, dst_base, split_name):
    for fname in file_list:
        shutil.copy(os.path.join(src_img_dir, fname),
                    os.path.join(dst_base, split_name, "images", fname))
        shutil.copy(os.path.join(src_mask_dir, fname),
                    os.path.join(dst_base, split_name, "masks", fname))

copy_pair_list(train, merged_img_dir, merged_mask_dir, split_base, "train")
copy_pair_list(val, merged_img_dir, merged_mask_dir, split_base, "val")
copy_pair_list(test, merged_img_dir, merged_mask_dir, split_base, "test")

print("Splitting done. Train:", len(train), "Val:", len(val), "Test:", len(test))


Total valid pairs available: 1086
Splitting done. Train: 814 Val: 163 Test: 109


In [21]:
# Install required libs
!pip install albumentations --quiet

import cv2, numpy as np, torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

class PolypSegDataset(Dataset):
    def __init__(self, img_dir, mask_dir, augment=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.files = sorted([f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
        self.augment = augment

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = cv2.imread(os.path.join(self.img_dir, self.files[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(os.path.join(self.mask_dir, self.files[idx]), 0)
        mask = (mask > 127).astype(np.float32)

        if self.augment:
            aug = self.augment(image=img, mask=mask)
            img, mask = aug['image'], aug['mask']

        img = ToTensorV2()(image=img)['image']
        mask = torch.from_numpy(mask).unsqueeze(0)
        return img, mask

# Augmentations
train_aug = A.Compose([
    A.Resize(256,256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.CLAHE(p=1.0),
    ToTensorV2()
])
val_aug = A.Compose([A.Resize(256,256), ToTensorV2()])

# Dataloaders
train_ds = PolypSegDataset("/content/combined_split2/train/images","/content/combined_split2/train/masks", augment=train_aug)
val_ds   = PolypSegDataset("/content/combined_split2/val/images"," /content/combined_split2/val/masks", augment=val_aug)
test_ds  = PolypSegDataset("/content/combined_split2/test/images","/content/combined_split2/test/masks", augment=val_aug)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=2)


In [22]:
!pip install segmentation-models-pytorch --quiet

import torch
import segmentation_models_pytorch as smp

# Use a lightweight backbone
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None,
).to(device)

# Loss and optimizer
loss_fn = smp.losses.DiceLoss(mode="binary")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [25]:
def train_one_epoch(epoch):
    model.train()
    total_loss = 0.0
    for imgs, masks in train_loader:
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        preds = model(imgs)
        loss = loss_fn(preds, masks)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    return avg_loss

def validate():
    model.eval()
    tot_loss, tot_batches = 0.0, 0
    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            loss = loss_fn(preds, masks)
            tot_loss += loss.item()
            tot_batches += 1
    return tot_loss / tot_batches

for epoch in range(1, 11):  # 10 epochs as a starting point
    tl = train_one_epoch(epoch)
    vl = validate()
    print(f"Epoch {epoch}: TrainDice/TrainLoss approx {tl:.4f}, ValLoss={vl:.4f}")


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipython-input-403634840.py", line 30, in __getitem__
    mask = torch.from_numpy(mask).unsqueeze(0)
           ^^^^^^^^^^^^^^^^^^^^^^
TypeError: expected np.ndarray (got Tensor)


In [24]:
# Re-run the data loading cell with the corrected augmentations
# This cell was modified to fix the ToTensorV2 placement
# cell_id: stGgqF8H9Yf4

# Dataloaders
train_ds = PolypSegDataset("/content/combined_split2/train/images","/content/combined_split2/train/masks", augment=train_aug)
val_ds   = PolypSegDataset("/content/combined_split2/val/images"," /content/combined_split2/val/masks", augment=val_aug)
test_ds  = PolypSegDataset("/content/combined_split2/test/images","/content/combined_split2/test/masks", augment=val_aug)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=8, shuffle=False, num_workers=2)

print("Dataloaders updated with corrected augmentations.")

Dataloaders updated with corrected augmentations.


In [26]:

import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class PolypSegDataset(Dataset):
    def __init__(self, img_dir, mask_dir, augment=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.files = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        self.augment = augment

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        img_path = os.path.join(self.img_dir, fname)
        mask_path = os.path.join(self.mask_dir, fname)

        image = cv2.imread(img_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Read mask as NumPy array
        mask_np = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask_np is None:
            raise FileNotFoundError(f"Mask not found: {mask_path}")
        mask_np = (mask_np > 127).astype(np.float32)  # [H,W] -> [0,1]

        if self.augment:
            augmented = self.augment(image=image, mask=mask_np)
            image = augmented['image']
            mask_np = augmented['mask']

        # Ensure final torch.tensor
        # image is already a tensor from ToTensorV2 in your augment, otherwise convert here:
        if not isinstance(image, torch.Tensor):
            image = torch.from_numpy(image).permute(2,0,1).float() / 255.0

        if isinstance(mask_np, np.ndarray):
            mask = torch.from_numpy(mask_np).unsqueeze(0)
        elif isinstance(mask_np, torch.Tensor):
            mask = mask_np.unsqueeze(0)
        else:
            raise TypeError("Mask must be a NumPy array or a Torch tensor at this point")

        return image, mask


In [27]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_aug = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.CLAHE(p=1.0),
    # Do not include ToTensorV2 here if you convert in dataset
], p=1.0)

val_aug = A.Compose([
    A.Resize(256, 256),
], p=1.0)


In [28]:
train_ds = PolypSegDataset("/content/combined_split2/train/images",
                         "/content/combined_split2/train/masks",
                         augment=train_aug)

val_ds = PolypSegDataset("/content/combined_split2/val/images",
                       "/content/combined_split2/val/masks",
                       augment=val_aug)

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=8, shuffle=False, num_workers=4)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
import torch
import segmentation_models_pytorch as smp

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
).to(device)

loss_fn = smp.losses.DiceLoss(mode="binary")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

def train_one_epoch(epoch):
    model.train()
    total_loss = 0.0
    for imgs, masks in train_loader:
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        preds = model(imgs)
        loss = loss_fn(preds, masks)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.item())
    return total_loss / len(train_loader)

def validate():
    model.eval()
    tot = 0.0
    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            loss = loss_fn(preds, masks)
            tot += float(loss.item())
    return tot / len(val_loader)

for epoch in range(1, 11):
    train_loss = train_one_epoch(epoch)
    val_loss = validate()
    print(f"Epoch {epoch} Train Loss: {train_loss:.4f}  Val Loss: {val_loss:.4f}")

# Save best model (example)
torch.save(model.state_dict(), "/content/drive/MyDrive/models/attention_unetpp.pth")
